<a href="https://colab.research.google.com/github/MichaelHuang05/test-repo/blob/master/FintechDR-CNN%20with%20resample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#from google.colab import files
#uploaded = files.upload()

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate


In [0]:
!mkdir -p Drive
!google-drive-ocamlfuse Drive

/bin/bash: google-drive-ocamlfuse: command not found


In [7]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

from google.colab import drive
drive.mount('/content/Drive')


path = "/content/Drive/My Drive/Colab Notebooks/images"
os.chdir(path) 
with open('/content/Drive/My Drive/Colab Notebooks/labels.json') as label:
    labels = json.load(label)

label_df = pd.DataFrame(labels, columns = ["file_name", "DR_stage"])


Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).


In [13]:
#### training and testing set split
from sklearn.model_selection import train_test_split 
from keras.utils import to_categorical


y_train, y_test = train_test_split(label_df, test_size=0.2)

q = pd.DataFrame(y_train)
a = q[q["DR_stage"].isin([0])]
b = q[q["DR_stage"].isin([1])]
c = q[q["DR_stage"].isin([2])]
d = q[q["DR_stage"].isin([3])]
e = q[q["DR_stage"].isin([4])]

q.groupby(['DR_stage']).agg({'DR_stage':"count"})
# d



Using TensorFlow backend.


,DR_stage
DR_stage,
0,881
1,99
2,177
3,28
4,15


In [0]:
### we need to bootstrap to make each level to have the "balanced" amount of observations 
from random import sample

boota = np.random.choice(a["file_name"], replace = True, size = 240)
bootb = np.random.choice(b["file_name"], replace = True, size = 240)
bootc = np.random.choice(c["file_name"], replace = True, size = 240)
bootd = np.random.choice(d["file_name"], replace = True, size = 240)
boote = np.random.choice(e["file_name"], replace = True, size = 240)

### later use boot_df
boota_df = pd.DataFrame(boota, columns = ["file_name"])
bootb_df = pd.DataFrame(bootb, columns = ["file_name"])
bootc_df = pd.DataFrame(bootc, columns = ["file_name"])
bootd_df = pd.DataFrame(bootd, columns = ["file_name"])
boote_df = pd.DataFrame(boote, columns = ["file_name"])


train_filenames_df = pd.concat([boota_df, bootb_df, bootc_df, bootd_df, boote_df])
y_new_train = pd.merge(train_filenames_df, label_df, how = "left", on = ["file_name", "file_name"])


# Onehot encoding for catogory
y_new_trainOneHot = to_categorical(y_new_train.iloc[:,1])
y_testOneHot = to_categorical(y_test.iloc[:,1])


In [0]:
# Train set
################################################

train_images = []
train_filenames_ls = list(train_filenames_df["file_name"])

for i in range(len(train_filenames_ls)):              
    temp = cv2.imread(path + "/" + train_filenames_ls[i])
    temp = cv2.resize(temp, (512,512))
    train_images.append(temp)

train_images_arr = np.array(train_images)

# Test set
#############################################

test_images = []
test_filenames_ls = list(y_test["file_name"])

for i in range(len(test_filenames_ls)):              
    temp = cv2.imread(path + "/" + test_filenames_ls[i])
    temp = cv2.resize(temp, (512,512))
    test_images.append(temp)

test_images_arr = np.array(test_images)


In [16]:
### CNN 
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D




model = Sequential()

model.add(Conv2D(filters = 10, kernel_size = (3,3), input_shape = (512,512,3), activation = "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(filters = 10, kernel_size = (3,3), activation = "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters = 10, kernel_size = (3,3), activation = "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters = 10, kernel_size = (3,3), activation = "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Flatten())
model.add(Dense(128, activation = "relu"))

model.add(Dropout(0.2))

model.add(Dense(128, activation = "relu"))

model.add(Dropout(0.2))

model.add(Dense(5, activation = 'softmax'))

model.compile(loss = "categorical_crossentropy",
             optimizer = "adam",
             metrics = ['accuracy'])

# maybe should change to focal loss?


#model = tf.contrib.tpu.keras_to_tpu_model(
#    model,
#    strategy=tf.contrib.tpu.TPUDistributionStrategy(
#        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))



model.summary()







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 510, 510, 10)      280       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 255, 255, 10)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 253, 253, 10)      910       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 126, 126, 10)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 124, 124, 10)      910       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 62, 62, 10)        0        

In [17]:
model.fit(x = train_images_arr, y = y_new_trainOneHot, batch_size = 100, epochs = 1000, validation_split = 0.1, verbose = 2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 1080 samples, validate on 120 samples
Epoch 1/1000





 - 16s - loss: 11.3264 - acc: 0.2417 - val_loss: 13.5272 - val_acc: 0.0000e+00
Epoch 2/1000
 - 5s - loss: 4.2867 - acc: 0.2750 - val_loss: 1.8704 - val_acc: 0.2000
Epoch 3/1000
 - 5s - loss: 1.5292 - acc: 0.3176 - val_loss: 1.6193 - val_acc: 0.2917
Epoch 4/1000
 - 5s - loss: 1.3661 - acc: 0.4370 - val_loss: 1.4729 - val_acc: 0.2917
Epoch 5/1000
 - 5s - loss: 1.1784 - acc: 0.4991 - val_loss: 1.1585 - val_acc: 0.4167
Epoch 6/1000
 - 5s - loss: 1.0066 - acc: 0.5972 - val_loss: 0.8676 - val_acc: 0.6917
Epoch 7/1000
 - 5s - loss: 0.8156 - acc: 0.6741 - val_loss: 0.3941 - val_acc: 0.9417
Epoch 8/1000
 - 5s - loss: 0.6490 - acc: 0.7676 - val_loss: 0.3403 - val_acc: 0.9417
Epoch 9/1000
 - 5s - loss: 0.5215 - acc: 0.8093 - val_loss: 0.2250 - val_acc: 0.9417
Epoch 10/1000
 - 5s - loss: 0.4367 - acc: 0.8343 - val_loss: 0.1795 - val_acc: 

In [18]:
scores = model.evaluate(test_images_arr, y_testOneHot)
scores[1]

300/300 [==============================] - 1s 4ms/step


0.41

In [19]:

prediction = model.predict_classes(test_images_arr)
pd.crosstab(y_test.iloc[:,1],prediction,
            rownames=['label'],colnames=['predict'])

predict,0,1,2,3,4
label,,,,,
0,97,39,66,11,1
1,9,1,9,2,0
2,22,4,25,1,0
3,1,0,6,0,0
4,1,0,4,1,0


In [22]:
#### classification report

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


names = ["stage 0", "stage 1", "stage 2", "stage 3", "stage 4"]
print(classification_report(y_test.iloc[:,1], prediction, target_names = names))
#print(confusion_matrix(y_test, prediction, labels=[0,1]))


              precision    recall  f1-score   support

     stage 0       0.75      0.45      0.56       214
     stage 1       0.02      0.05      0.03        21
     stage 2       0.23      0.48      0.31        52
     stage 3       0.00      0.00      0.00         7
     stage 4       0.00      0.00      0.00         6

    accuracy                           0.41       300
   macro avg       0.20      0.20      0.18       300
weighted avg       0.57      0.41      0.46       300

